# **Promedio de valor de Créditos para casas nueva y para vivienda usada**

In [8]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "tulancingo" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_tulancingo',
 'df_financiamientos_2020_tulancingo',
 'df_financiamientos_2021_tulancingo',
 'df_financiamientos_2022_tulancingo',
 'df_financiamientos_2023_tulancingo',
 'df_financiamientos_2024_tulancingo']

## 2022

In [2]:
finance_2022 = df_financiamientos_2022_tulancingo[['id','mes','modalidad','monto']]
shape = finance_2022.shape
print(shape)
print(finance_2022.head())

(316, 4)
   id  mes  modalidad      monto
0   1    1          2       0.00
1   2    1          2       0.00
2   3    1          2   14398.00
3   4    1          4   96283.00
4   5    1          3  159720.57


In [4]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2022['modalidad'] = finance_2022['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2022.loc[~finance_2022['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2022 = finance_2022[finance_2022['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2022['trimestre'] = (finance_2022['mes'] - 1) // 3 + 1
finance_2022

,id,mes,modalidad,monto,trimestre
4,5,1,Usada,159720.57,1
5,6,1,Nueva,241963.35,1
7,8,1,Nueva,369616.07,1
8,9,1,Usada,388575.19,1
9,10,1,Nueva,572183.25,1
...,...,...,...,...,...
311,312,12,Usada,2279250.00,4
312,313,12,Usada,2285795.72,4
313,314,12,Nueva,2661350.00,4
314,315,12,Usada,2946484.00,4


In [5]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2022.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.map('{:,.2f}'.format)

# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2022',
    2: '2T 2022',
    3: '3T 2022',
    4: '4T 2022'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2022,2T 2022,3T 2022,4T 2022
0,Nueva,"1,026,008.53","1,071,404.65","896,403.19","943,715.36"
1,Usada,"754,953.58","1,088,308.40","931,491.31","1,068,239.56"


In [6]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2022,"1,026,008.53"
4,Nueva,2T 2022,"1,071,404.65"
6,Nueva,3T 2022,"896,403.19"
8,Nueva,4T 2022,"943,715.36"
3,Usada,1T 2022,"754,953.58"
5,Usada,2T 2022,"1,088,308.40"
7,Usada,3T 2022,"931,491.31"
9,Usada,4T 2022,"1,068,239.56"


In [7]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=f'{df_modalidad.iloc[i]["promedio"]}',
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_creditos_nuevausada_2022', carpeta='assets\graficas')
fig.show()

## 2023

In [7]:
finance_2022 = df_financiamientos_2023_tulancingo[['id','mes','modalidad','monto']]
shape = finance_2022.shape
print(shape)
print(finance_2022.head())

(45, 4)
   id  mes  modalidad       monto
0   1    1          1  1585485.28
1   2    1          1   608908.05
2   3    1          1  1918694.86
3   4    1          1  1241000.00
4   5    2          1        0.00


In [8]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2022['modalidad'] = finance_2022['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2022.loc[~finance_2022['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2022 = finance_2022[finance_2022['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2022['trimestre'] = (finance_2022['mes'] - 1) // 3 + 1
finance_2022

C:\Users\yoe11\AppData\Local\Temp\ipykernel_23224\219771975.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
0,1,1,Nueva,1585485.28,1
1,2,1,Nueva,608908.05,1
2,3,1,Nueva,1918694.86,1
3,4,1,Nueva,1241000.00,1
4,5,2,Nueva,0.00,1
5,6,2,Nueva,390087.63,1
6,7,2,Nueva,604900.00,1
7,8,2,Nueva,1429200.00,1
8,9,2,Nueva,1900000.00,1
9,10,2,Nueva,1100000.00,1


In [9]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2022.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.map('{:,.2f}'.format)

# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2023',
    2: '2T 2023',
    3: '3T 2023',
    4: '4T 2023'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2023,2T 2023,3T 2023
0,Nueva,"1,072,762.53","1,031,834.45","1,092,332.23"


In [10]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
1,Nueva,1T 2023,"1,072,762.53"
2,Nueva,2T 2023,"1,031,834.45"
3,Nueva,3T 2023,"1,092,332.23"


In [11]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=f'{df_modalidad.iloc[i]["promedio"]}',
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_creditos_nuevausada_2023', carpeta='assets\graficas')
fig.show()

## 2024

In [ ]:
finance_2022 = df_financiamientos_2024_tulancingo[['id','mes','modalidad','monto']]
shape = finance_2022.shape
print(shape)
print(finance_2022.head())

(45, 4)
   id  mes  modalidad       monto
0   1    1          1  1585485.28
1   2    1          1   608908.05
2   3    1          1  1918694.86
3   4    1          1  1241000.00
4   5    2          1        0.00


In [ ]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2022['modalidad'] = finance_2022['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2022.loc[~finance_2022['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2022 = finance_2022[finance_2022['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2022['trimestre'] = (finance_2022['mes'] - 1) // 3 + 1
finance_2022

C:\Users\yoe11\AppData\Local\Temp\ipykernel_23224\219771975.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
0,1,1,Nueva,1585485.28,1
1,2,1,Nueva,608908.05,1
2,3,1,Nueva,1918694.86,1
3,4,1,Nueva,1241000.00,1
4,5,2,Nueva,0.00,1
5,6,2,Nueva,390087.63,1
6,7,2,Nueva,604900.00,1
7,8,2,Nueva,1429200.00,1
8,9,2,Nueva,1900000.00,1
9,10,2,Nueva,1100000.00,1


In [ ]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2022.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.map('{:,.2f}'.format)

# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2023',
    2: '2T 2023',
    3: '3T 2023',
    4: '4T 2023'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2023,2T 2023,3T 2023
0,Nueva,"1,072,762.53","1,031,834.45","1,092,332.23"


In [ ]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
1,Nueva,1T 2023,"1,072,762.53"
2,Nueva,2T 2023,"1,031,834.45"
3,Nueva,3T 2023,"1,092,332.23"


In [ ]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=f'{df_modalidad.iloc[i]["promedio"]}',
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_creditos_nuevausada_2024', carpeta='assets\graficas')
fig.show()